In [1]:
import clickhouse_connect
import pandas as pd
import polars as pl

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Direct connect to Cryptohouse
Connect to cryptohouse with client to do some initial querying and result aggregation.


### Limitations
- Can only do 1000 blocks, the direct access limits are still being figured out.


In [2]:
# Create a connection to the ClickHouse server
client = clickhouse_connect.get_client(
    host='crypto-clickhouse.clickhouse.com',
    username='crypto',
    password='',
    secure=True
)

In [9]:
# Execute the query and return as a pandas dataframe
query: str = """
SELECT
  *
FROM
  solana.transactions ARRAY
  JOIN accounts AS account
WHERE
  arrayExists(
    x -> x.1 IN [ 
        'Fc8bpeCMifWYv97pQ3k5xDvd98nuVg6yAaZrwmy4RRp6', -- zeta dex address
        'pythWSnswVUd12oZpeFP8e9CVaEqJg25g1Vtc2biRsT' -- pyth price feed
    ],
    accounts
  )
 // The block_slot range 282,300,000 to 282,900,000 is about 3 days and is a known range where probems were being experienced
 AND block_slot > 282899000
 AND block_slot < 282900000
  // AND block_timestamp >= now() - INTERVAL 5 MINUTE
  AND account.2 = true -- Filter for signer accounts only
"""

In [10]:
results: pd.DataFrame = client.query_df(query)

In [11]:
df = pl.from_pandas(results)

In [12]:
df.group_by('status', 'log_messages').agg(
    pl.len().alias('count')).sort(by='count', descending=True)

status,log_messages,count
str,list[str],u32
"""0""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD failed: custom program error: 0x1819""]",702
"""1""","[""Program 11111111111111111111111111111111 invoke [1]"", ""Program 11111111111111111111111111111111 success"", … ""Program ComputeBudget111111111111111111111111111111 success""]",410
"""0""","[""Program pythWSnswVUd12oZpeFP8e9CVaEqJg25g1Vtc2biRsT invoke [1]"", ""Program log: Instruction: UpdatePriceFeed"", … ""Program pythWSnswVUd12oZpeFP8e9CVaEqJg25g1Vtc2biRsT failed: custom program error: 0x1777""]",134
"""1""","[""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ invoke [1]"", ""Program log: Instruction: CloseEncodedVaa"", … ""Program ComputeBudget111111111111111111111111111111 success""]",82
"""0""","[""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ invoke [1]"", ""Program log: Instruction: WriteEncodedVaa"", … ""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ failed: custom program error: 0xbbb""]",82
…,…,…
"""1""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD success""]",1
"""1""","[""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD invoke [1]"", ""Program log: Instruction: UpdatePricingV3"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD success""]",1
"""1""","[""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ invoke [1]"", ""Program log: Instruction: WriteEncodedVaa"", … ""Program ComputeBudget111111111111111111111111111111 success""]",1


In [13]:
# List of jito tip accounts
jito_tip_accts = [
    '96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5',
    'HFqU5x63VTqvQss8hp11i4wVV8bD44PvwucfZ2bU7gRe',
    'Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY',
    'ADaUMid9yfUytqMBgopwjb2DTLSokTSzL1zt6iGPaS49',
    'DfXygSm4jCyNCybVYYK6DwvWqjKee8pbDmJGcLWNDXjh',
    'ADuUkR4vqLUMWXxW9gh6D6L8pMSawimctcNZ5pGwDcEt',
    'DttWaMuVvTiduZRnguLF7jNxTgiMBZ1hyAumKUiL2KRL',
    '3AVi9Tg9Uo68tJfuvoKvqKNWKkC5wPdSSdeBnizKZ6jT'
]

In [48]:
df.select('block_slot').unique().shape

(511, 1)

In [50]:
(
    df.explode('accounts')
    .with_columns(
    pl.col("accounts").struct.field("pubkey").alias("jit_acct")
    )
).filter(pl.col('jit_acct').is_in(jito_tip_accts)).select('block_slot', 'accounts', 'jit_acct').unique()

block_slot,accounts,jit_acct
i64,struct[3],str
282899272,"{""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"",false,true}","""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"""
282899889,"{""DfXygSm4jCyNCybVYYK6DwvWqjKee8pbDmJGcLWNDXjh"",false,true}","""DfXygSm4jCyNCybVYYK6DwvWqjKee8pbDmJGcLWNDXjh"""
282899088,"{""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"",false,true}","""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"""
282899213,"{""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"",false,true}","""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"""
282899793,"{""DttWaMuVvTiduZRnguLF7jNxTgiMBZ1hyAumKUiL2KRL"",false,true}","""DttWaMuVvTiduZRnguLF7jNxTgiMBZ1hyAumKUiL2KRL"""
…,…,…
282899257,"{""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"",false,true}","""Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY"""
282899205,"{""HFqU5x63VTqvQss8hp11i4wVV8bD44PvwucfZ2bU7gRe"",false,true}","""HFqU5x63VTqvQss8hp11i4wVV8bD44PvwucfZ2bU7gRe"""
282899842,"{""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"",false,true}","""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"""
